In [4]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

# Face Detection Model
- Load the model

In [5]:
face_detection_model = './models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'
# Load models using cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
description_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [2]:
!dir emotion_data

 Volume in drive C is OS
 Volume Serial Number is 30EA-83F7

 Directory of C:\Face Recogniation App\Face-Recognition\5_face_recognition_emotion\emotion_data

11/24/2021  10:03 PM    <DIR>          .
11/24/2021  10:03 PM    <DIR>          ..
11/24/2021  10:02 PM    <DIR>          angry
11/24/2021  10:03 PM    <DIR>          disgust
11/24/2021  10:03 PM    <DIR>          fear
11/24/2021  10:03 PM    <DIR>          happy
11/24/2021  10:03 PM    <DIR>          neutral
11/24/2021  10:03 PM    <DIR>          sad
11/24/2021  10:03 PM    <DIR>          surprise
               0 File(s)              0 bytes
               9 Dir(s)  285,041,741,824 bytes free


In [9]:
# consider sample image
img = cv2.imread('./emotion_data/angry/0.jpg')
cv2.imshow('Angry',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# step-1: face detection 
# image =  img.copy()
# h,w = image.shape[:2]
# img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop =False)
# set the input
# detector_model.setInput(img_blob)
# detections = detector_model.forward()

# if len(detections) >0:
#     i = np.argmax(detections[0,0,:,2]) # consider the face with max confidence score 
#     confidence = detections[0,0,i,2]
#     if confidence >0.5:
#         box = detections[0,0,i,3:7]*np.array([w,h,w,h])
#         (startx,starty,endx,endy) = box.astype('int')
        # just for reference: draw bounding 
#         img_draw = image.copy()
#         cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))

#         # step -2: feauture extraction or embedding
#         roi = image[starty:endy,startx:endx].copy()
#         # get the face descriptors
#         faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
#         description_model.setInput(faceblob)
#         vectors = description_model.forward()
#         print(vectors)
        
# cv2.imshow('sample',img)
# cv2.imshow('draw_image',img_draw)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
def helper(image_path):
    img = cv2.imread(image_path)
    # step-1: face detection 
    image =  img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop =False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) >0:
        i = np.argmax(detections[0,0,:,2]) # consider the face with max confidence score 
        confidence = detections[0,0,i,2]
        if confidence >0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # just for reference: draw bounding 
    #         img_draw = image.copy()
    #         cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))

            # step -2: feauture extraction or embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB = True,crop = True)
            description_model.setInput(faceblob)
            vectors = description_model.forward()
            
            return vectors # if the face is detected or None
        
    return None


In [14]:
helper('./emotion_data/angry/0.jpg')

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [12]:
data = dict(data=[], label=[])

# apply helper function to all images and get face descriptors

In [13]:
folders = os.listdir('emotion_data')
for folder in folders:
    filenames = os.listdir('emotion_data/{}'.format(folder))
    for filename in filenames:
        try:
            vector = helper('./emotion_data/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature extracted successfully')
                
        except:
            pass
                        

Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature 

Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature extracted successfully
Feature 

limit_output extension: Maximum message size of 10000 exceeded with 10044 characters

In [15]:
data.keys()

dict_keys(['data', 'label'])

In [16]:
pd.Series(data['label']).value_counts()

neutral     292
sad         260
happy       244
fear        207
angry       189
surprise    181
disgust      40
dtype: int64

In [17]:
# save the data 
pickle.dump(data,open('data_face_features_emotion.pickle',mode='wb'))